In [ ]:
import pandas
import neurolab as nl
import numpy as np
import os
# Calculate accuracy using sMAPE

def calMAPE(X_test,Y_test,sigma,mu,result):
	sum_up = 0
	n = 0
	size = len(X_test)
	for i in range(size):
		# Find the trips of the bus stop that has arrival time of previous stop
		# to ensure our accuracy testing is valid
		diff = (result[i] - Y_test[i])
		diff = abs(diff)
		Deno = abs(result[i]) + abs(Y_test[i])
		n = n+1
		sum_up = sum_up + (diff/Deno)
	# sMAPE is calculated. Named as MAPE since no need to worry about this, the calculation is changed.
	MAPE = sum_up/n
		#print(MAPE)
	return MAPE

def CrossV(X,Y,n1,n2,sigma,mu):
	mapeTotal = 0
	# Divide the data into 5 fold
	fold = np.random.random_integers(1,5,len(X))
	index = np.array([1,2,3,4,5])
	# 5-fold CV
	t1 = 'CV Starts'
	fout.write(t1)
	fout.write('\n')
	for i in index:
   		test_index = np.where(fold == i)
		test_index = test_index[0]
		train_index = np.where(fold != i)
		train_index = train_index[0]
		X_train = X[train_index,:]
		X_test = X[test_index,:]
		y_train = Y[train_index,:1]
		y_test = Y[test_index,:1]
		# Set the function to newff
		net = nl.net.newff([[-5,5]]*11,[n1,n2,1],[nl.trans.LogSig(), nl.trans.LogSig(), nl.trans.PureLin()])
		# Add bias parameter 1 to input layer and hidden layer
		net.layers[0].np['b'][:] = 1
		net.layers[1].np['b'][:] = 1
		net.layers[2].np['b'][:] = 1
		# Set training function to Conjugated Gradient
		net.trainf = nl.train.train_cg
		# Set error function to mean absolute error
		net.errorf = nl.error.MSE()
		# Train neuro network
		error = net.train(X_train, y_train, epochs=2000, goal=0.0001)
		# Get predicted time from test set
		result = net.sim(X_test)
		# Calculate accuracy with MAPE
		thisMape = calMAPE(X_test,y_test,sigma,mu,result)
		#print thisMape
		mapeTotal = mapeTotal + thisMape
		mm = thisMape
		if isinstance(mm,np.ndarray):
			mm = mm[0]
		tt = repr(n1) + ',' + repr(n2) + ',' + repr(mm)
		fout.write(tt)
		fout.write('\n')
	mapeCV = mapeTotal/5
	t2 = 'CV Ends'
	fout.write(t2)
	fout.write('\n')
	# Return the mape value and the parameters
	return mapeCV, n1, n2

# Find the best parameter
def findbest(X_i,Y_i,sigma,mu):
	# Set the range of the unit values in layer 1
	n1 = np.arange(5,8,1)
	# Set the range of the unit values in layer 2
	n2 = np.arange(3,6,1)
	# Initialize the accuracy and n2 array to store the best n2 and accuracy
	# at each n1
	Bestn2Array = np.array(0)
	BestAccArray = np.array(0)
	for i in n1:
		# Initialize the accuracy and n2 array to store the n2 values that
		# have been tested and their accuracy at each n1
		FindBestn2 = np.array(0)
		FindBestAcc = np.array(0)
		for j in n2:
			# Get accuracy(MAPE at this time) from 5-fold CV
			Acc_i,n1_i,n2_i = CrossV(X_i,Y_i,i,j,sigma,mu)
			# Store the n2 and accuracy(MAPE) values
			FindBestn2 = np.append(FindBestn2, n2_i)
			FindBestAcc = np.append(FindBestAcc, Acc_i)
		# Delete the initializing value of the arrays
		FindBestn2 = np.delete(FindBestn2,[0])
		FindBestAcc = np.delete(FindBestAcc,[0])
		# Find the minimum accuracy as now it's still MAPE which is 1-accuracy
		BestAcc_i = np.amin(FindBestAcc) 
		# Find the n2 that correspond to the minimum MAPE
		Bestn2_i = FindBestn2[np.argmin(FindBestAcc)]
		# Store the best n2 and accuracy correspond to every n1 value
		Bestn2Array = np.append(Bestn2Array,Bestn2_i)
		BestAccArray = np.append(BestAccArray,BestAcc_i)
	# Delete the initializing value 0
	Bestn2Array = np.delete(Bestn2Array,[0])
	BestAccArray = np.delete(BestAccArray,[0])
	print(Bestn2Array)
	print(BestAccArray)
	# Find the minimum MAPE(best accuracy)
	BestAcc = np.amin(BestAccArray)
	# Get the corresponding n2 and n1
	Bestn2 = Bestn2Array[np.argmin(BestAccArray)]
	Bestn1 = n1[np.argmin(BestAccArray)]
	# Get the accuracy 
	BestAccuracy_i = BestAcc
	text = repr(Bestn1) + ',' + repr(Bestn2) + ',' + repr(BestAccuracy_i)
	fout.write(text)
	fout.write('\n')
	return Bestn1, Bestn2, BestAccuracy_i

for t in ['1','2','3','4','5','6','7','8','9','10']:
    fn = './output'+t+'.txt'
    if os.path.exists(fn):
        os.remove(fn) 

    # Import data
    Data = r'./../Data3/busStop/bustStop_'+t+'.csv'
    Dif = r'./../Data3/ExtraData/extraData_'+t+'.csv'
    mu = r'./../Data3/mu/mu_'+t+'.csv'
    sigma = r'./../Data3/sigma/sigma_'+t+'.csv'
    # Read data with panadas package
    mu = pandas.read_csv(mu)
    mu = float(mu.columns.values[0])
    sigma = pandas.read_csv(sigma)
    sigma = float(sigma.columns.values[0])

    X = pandas.read_csv(Data, names = ['a','b','c','d','e','f','g','h','i','j','k']);
    X = X.values
    Y = pandas.read_csv(Dif, names = ['diff']);
    Y = Y.values
    # Divide the whole training data into 5 subsets
    fold = np.random.random_integers(1,5,len(X))
    index = np.array([1,2,3,4,5])

    fout = open('./output'+t+'.txt', "a", 0)
    text1 = 'n1, tn2, Accuracy'
    fout.write(text1)
    fout.write('\n')

    # initialize accuracy array
    BestAccuracy = np.array(0)

    for i in index:
        test_index = np.where(fold == i)
        test_index = test_index[0]
        train_index = np.where(fold != i)
        train_index = train_index[0]
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = Y[train_index,:1]
        y_test = Y[test_index,:1]

        # Find the best units for the first and second layer of neural network
        # of each subset with 5-fold CV

        text5 = 'Evluating'
        fout.write(text5)
        fout.write('\n')
        n1_i, n2_i, Accuracy_i = findbest(X_train,y_train,sigma,mu)
        # Call neural network training function and calculate MAPE value of each
        # subset with the best parameter found
        EvaluationArray = np.arange(1,11,1)
        BestMape = 200
        for k in EvaluationArray:
            net = nl.net.newff([[-5,5]]*(11),[n1_i,n2_i,1],[nl.trans.LogSig(),nl.trans.LogSig(), nl.trans.PureLin()])
            net.layers[0].np['b'][:] = 1
            net.layers[1].np['b'][:] = 1
            net.layers[2].np['b'][:] = 1
            net.trainf = nl.train.train_cg
            net.errorf = nl.error.MSE()
            error = net.train(X_train, y_train, epochs=2000, goal=0.0001)
            result = net.sim(X_test)
            Mape = calMAPE(X_test,y_test,sigma,mu,result)
            # Get accuracy with sMAPE
            if Mape < BestMape:
                BestMape = Mape
                resultFinal = result
                y_testFinal = y_test
                net.save('BusStop13.net')
            Accu_i = BestMape


        # Store accuracy of each subset to BestAccuracy
        BestAccuracy = np.append(BestAccuracy,Accu_i)
        print(Accu_i)
        text6 = 'Evluating Ends'
        fout.write(text6)
        fout.write('\n')
        R = 'Results = ' + repr(resultFinal)
        fout.write(R)
        fout.write('\n')
        A = 'Actual = ' + repr(y_testFinal)
        fout.write(A)
        fout.write('\n')



    # Delete initializing value 0
    BestAccuracy = np.delete(BestAccuracy,[0])
    # Get average accuracy of the 5 fold
    sMAPE = np.average(BestAccuracy)
    # Find best parameter for the entire dataset
    n1,n2,Acc = findbest(X,Y,sigma,mu)
    
    print('for bus stop {0}').format(t)
    print('sMAPE:',sMAPE)
    print('Units for the first layer:',n1)
    print('Units for the second layer:',n2)
    text2 = 'Final Bestn1, Bestn2, Accuracy'
    fout.write(text2)
    fout.write('\n')
    text3 = repr(n1) + ',' + repr(n2) + ',' + repr(sMAPE)
    fout.write(text3)
    fout.write('\n')
    fout.close
